In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout, Dense, Flatten, Reshape, Conv1D, MaxPool1D, TimeDistributed, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from code.modelhelper import *

c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\program files\python37\lib\site-packages\tensorboard\compat\tensorflow_stub\dty

In [2]:
CVECT = 100
UCHARS = 139
NCLASS = 2
BATCH_SIZE = 192
EPOCHS = 300

In [3]:
x = np.load("data/xy/x_char_onehot_45_15.npz")["arr_0"]
y = np.load("data/xy/y_onehot_45_15.npy")
x.shape, y.shape

((60000, 13900), (60000, 2))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=11
)

# Char-level CNN

In [5]:
model_input = Input(shape=(CVECT*UCHARS,), name="m2_input")
H = Reshape((CVECT, UCHARS, 1), name="m2_reshape")(model_input)

H = TimeDistributed(
    Conv1D(
        filters=20, 
        kernel_size=(2), 
        padding='same', 
        activation='tanh',
        name="m2_conv1d_1"
    ),
    name="m2_timedistributed_1"
)(H)
H = TimeDistributed(MaxPool1D(2, name="m2_maxpool1d_1"), name="m2_timedistributed_2")(H)

H = TimeDistributed(
    Conv1D(
        filters=20, 
        kernel_size=(2), 
        padding='same', 
        activation='tanh',
        name="m2_conv1d_2"
    ),
    name="m2_timedistributed_3"
)(H)
H = TimeDistributed(MaxPool1D(2, name="m2_maxpool1d_2"), name="m2_timedistributed_4")(H)

H = TimeDistributed(
    Conv1D(
        filters=20, 
        kernel_size=(2), 
        padding='same', 
        activation='tanh',
        name="m2_conv1d_3"
    ),
    name="m2_timedistributed_5"
)(H)
H = TimeDistributed(MaxPool1D(2, name="m2_maxpool1d_3"), name="m2_timedistributed_6")(H)

H = TimeDistributed(Flatten(name="m2_flatten"), name="m2_timedistributed_7")(H)
H = Dropout(0.5, name="m2_dropout_1")(H)
H = GRU(256, return_sequences=True, name="m2_gru_1")(H)
H = GRU(256, return_sequences=False, go_backwards=True, name="m2_gru_2")(H)
H = Dropout(rate=0.5, name="m2_dropout_2")(H)
H = Dense(64, activation='tanh', name="m2_dense_1")(H)
model_output = Dense(2, activation='softmax', name="m2_output")(H)

model = Model(model_input, model_output)
model.summary()

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
m2_input (InputLayer)        [(None, 13900)]           0         
_________________________________________________________________
m2_reshape (Reshape)         (None, 100, 139, 1)       0         
_________________________________________________________________
m2_timedistributed_1 (TimeDi (None, 100, 139, 20)      60        
_________________________________________________________________
m2_timedistributed_2 (TimeDi (None, 100, 69, 20)       0         
_________________________________________________________________
m2_timedistributed_3 (TimeDi (None, 100, 69, 20)       820       
_________________________________________________________________
m2_timedistributed_4 (TimeDi (None, 100, 34, 20)       0         
_________________________________________________________________
m2_timedistributed_5 (TimeDi (None, 100, 34, 20)       820   

# Training

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(x_test, y_test),
    callbacks=[es]
)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred_model = model.predict(x_test)
y_pred = to_bin(y_pred_model)
print("Test recall: {}".format(recall(y_test, y_pred)))

y_test0 = to_1D(y_test)
y_pred0 = to_1D(y_pred)
print(confusion_matrix(y_test0, y_pred0))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history["accuracy"], label="Train")
plt.plot(history.history["val_accuracy"], label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
model.save("models/dl/m2.h5")

# K-fold CV

In [6]:
model_params = {
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "es": es,
}

In [7]:
scores = kfold(model, model_params, x, y, cv=10)

Starting 10-fold cross validation:
    Validation 01 of 10 ... done [810.17s]
    Validation 02 of 10 ... done [338.37s]
    Validation 03 of 10 ... done [360.55s]
    Validation 04 of 10 ... done [394.14s]
    Validation 05 of 10 ... done [377.98s]
    Validation 06 of 10 ... done [399.10s]
    Validation 07 of 10 ... done [464.22s]
    Validation 08 of 10 ... done [494.42s]
    Validation 09 of 10 ... done [393.64s]
    Validation 10 of 10 ... done [596.76s]


In [8]:
arr = scores_dict_to_array(scores)
arr

array([[0.97133333, 0.98333333, 0.98233333, 0.98183333, 0.986     ,
        0.98616667, 0.9905    , 0.99166667, 0.99333333, 0.99483333],
       [0.95391061, 0.98872445, 0.97666438, 0.98811189, 0.98754325,
        0.97560976, 0.99663753, 0.99191919, 0.98846893, 0.99511855],
       [0.92798913, 0.94351042, 0.95183946, 0.93887043, 0.9557937 ,
        0.96839274, 0.96610169, 0.97485109, 0.98594249, 0.98345968],
       [0.01457597, 0.00354531, 0.00754717, 0.00378198, 0.00399379,
        0.00797696, 0.00111957, 0.0026732 , 0.00405862, 0.0015388 ]])

In [9]:
np.save("cv/m2_cv.npy", arr)